Build a regression model.

In [ ]:
# imports
import statsmodels.api as sm
import pandas as pd
import numpy as np
import scipy

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
full_df = pd.read_csv('../data/bike_stations_hamilton_full_poi.csv', index_col=0)

grouped_by_staion = full_df[['station_id', 'empty_slots','free_bikes','review_count', 'rating', 'price']].groupby(by='station_id').mean()

grouped_by_staion.head()

In [ ]:
grouped_by_staion.info()

In [ ]:
# filling with 0 the two bike stations without any poi
grouped_by_staion[['review_count', 'rating']] = grouped_by_staion[['review_count', 'rating']].fillna(0)

In [ ]:
sns.pairplot(data=grouped_by_staion)
plt.show()

Provide model output and an interpretation of the results. 

In [ ]:
y = grouped_by_staion['free_bikes']
X = grouped_by_staion[['review_count', 'rating', 'price']]
X = sm.add_constant(X) #adds a column of 1's so the model will contain an intercept
X.head()

In [ ]:
model = sm.OLS(y, X) #instantiate
results = model.fit() #fit the model (this is where OLS is actually being run)
print(results.summary())

# Stretch

How can you turn the regression model into a classification model?